# Classificatore Random Forest

In [ ]:
# Numero di fold
num_folds = 10

# Inizializzazione delle liste per i valori di accuracy, F1 e valori di feature importance
accuracy_list = []
fold_f1_scores= []
all_importances= []

# Ciclo sui fold
for i in range(num_folds):
   
    # Lettura dei dati di addestramento di test e training
    train_data = pd.read_csv(f"kfold_Combo_and_Difference/fold_{i}/train.tsv", sep='\t')
    test_data = pd.read_csv(f"kfold_Combo_and_Difference/fold_{i}/test.tsv", sep='\t')
    
    # Estrazione delle feature e delle labels per test e training
    X_train = train_data.drop("Order_Label", axis=1)
    y_train = train_data["Order_Label"]
    X_test = test_data.drop("Order_Label", axis=1)
    y_test = test_data["Order_Label"]
    
    # Estrazione nomi simbolici delle feature
    features_names = X_train.columns.tolist()
    
    # Scaling valori con MinMaxScaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_fold, y_train_fold = X_train_scaled, y_train
    X_test_fold, y_test_fold = X_test_scaled, y_test
    
    # Creazione del classificatore Random Forest
    clf = RandomForestClassifier()

    # Addestramento del classificatore su dati di training e test del fold corrente
    clf.fit(X_train_fold, y_train_fold)

    # Testing del classificatore sul test set per il fold corrente
    fold_accuracy = clf.score(X_test_fold, y_test_fold)
    # Inserimento l'accuracy corrente nella lista delle accuracy
    accuracy_list.append(fold_accuracy)
    
    # Estrazione dei valori di feature importance
    fold_importance= clf.feature_importances_ 
    # Definizione di un dizionario che ha come chiave il nome della feature e come valore il corrispettivo
    feature_importances = dict(zip(features_names, fold_importance))
    # Inserimento del dizionario nella lista apposita per tutti i folds
    all_importances.append(feature_importances)
    
    # Estrazione predizioni del modello
    y_pred = clf.predict(X_test_fold)
    
    # Estrazione ed inserimento dei valori dell'F1 nella lista apposit
    fold_f1_score = f1_score(y_test_fold, y_pred, average='weighted')
    fold_f1_scores.append(fold_f1_score)
    
    print("Iterazione",i)
    
    print('Classification report:')
    # Stampa classification report
    print(classification_report(y_test_fold, y_pred))
    # Calcolo della media pesata dell'F1 score per ogni classe
    test_f1_score = f1_score(y_test_fold, y_pred, average='weighted')
    
    print('Confusion matrix:')
    cm = confusion_matrix(y_test_fold, y_pred)
    # Mostra Confusion Matrix del fold
    cm_display = ConfusionMatrixDisplay(cm)
    cm_display.display_labels = clf.classes_
    cm_display.plot(cmap='Blues', values_format='.4g')
    plt.title(f"Confusion Matrix for Fold {i+1}")
    plt.show()
    
    print('Feature più utili per discriminare le classi')
    
    # Ordina le feature in base al loro valore e prende le prime 20
    sorted_feature_importance = dict(sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)[:20])
    keys = list(sorted_feature_importance.keys())
    values = list(sorted_feature_importance.values())
    # Crea un grafico a barre
    plt.bar(keys, values)
    # Aggiunge un titolo e le etichette degli assi
    plt.title("Feature Importance")
    plt.xlabel("Feature")
    plt.ylabel("Importance")
    plt.xticks(rotation=90)
    plt.show()
    
print ("Risultati medi su tutti i fold")
# Calcolo dell'accuracy e f1 media
mean_accuracy = np.mean(accuracy_list)
mean_f1_score=np.mean(fold_f1_scores)

# Stampa dell'accuracy e f1 media
print(f"Mean Accuracy: {mean_accuracy}")
print(f"Mean F1: {mean_f1_score}")  

print("Feature importance:")

# Dizionario in cui salvare le medie dei valori per ogni chiave
averages_dict = {}

# Ciclo sui dizionari nella lista dei dizionari per ogni fold
for d in all_importances:
    # Ciclo sulle chiavi di ogni dizionario
    for key in d.keys():
        # Se la chiave non è presente nel dizionario delle medie, la aggiunge con valore 0
        if key not in averages_dict:
            averages_dict[key] = 0
        # Aggiorna la somma parziale dei valori per la chiave corrente
        averages_dict[key] += d[key]

# Divide le somme parziali per il numero di dizionari nella lista per ottenere la media
n = len(all_importances)
averages_dict = {key: value / n for key, value in averages_dict.items()}

sorted_feature_importances = dict(sorted(averages_dict.items(), key=lambda item: item[1], reverse=True))
    
# Salvo nelle variabili i nomi e i valori delle prime e ultime 15 feature discriminanti per la classificazione
top_features = list(sorted_feature_importances.keys())[:15] + list(sorted_feature_importances.keys())[-15:]
top_coef_values = list(sorted_feature_importances.values())[:15] + list(sorted_feature_importances.values())[-15:]


colors = ['skyblue' for val in top_coef_values]
# Plot delle coppie chiave-valore in ordine decrescente
plt.figure(figsize=(15, 5))
plt.bar(top_features, top_coef_values, color=colors, edgecolor='black')
plt.xlabel('Feature')
plt.ylabel('Importanza')
plt.xticks(rotation=90, fontsize=10)
plt.title('Significatività delle feature per la classificazione')
plt.grid(axis='x', alpha=0.4)
# Crea una lista di oggetti Patch per la legenda
legend_patches = [Patch(facecolor='darkblue', edgecolor='black', label='Significatività minore'), Patch(facecolor='skyblue', edgecolor='black', label='Significatività maggiore')]
# Crea la legenda personalizzata
plt.legend(handles=legend_patches)
plt.show()